In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'dreviews_2'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


dataset = load_dataset('ohidaoui/darija-reviews')


df = pd.DataFrame(dataset['test'])

display(len(df))
      
display(df.columns)
display(df[:4])

df['label'] = df['label'].replace('negative ', 'negative')

c = df['label'].value_counts()
display(c)

classes = set(df['label'].values)
display(classes)

df['label'] = df['label'].astype('category')
df['label'] = df['label'].cat.codes

df = df[['review', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))



max_sequence_length = 128



models = [ 
        'faisalq/EgyBERT',            
    'faisalq/SaudiBERT',            
    'tunis-ai/TunBERT',
    'alger-ia/dziribert',
    'SI2M-Lab/DarijaBERT',
    'otmangi/MorRoBERTa',
    'otmangi/MorrBERT'
            
]


seeds = [0, 1, 42]

for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
            
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['review'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True)
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')   
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{f1}\n')
                return {'accuracy': acc, 'f1_score': f1}
    
    
            
            
            epochs = 30
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 10, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 10
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                # data_collator=data_collator,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



Found cached dataset parquet (/home/ffq/.cache/huggingface/datasets/ohidaoui___parquet/ohidaoui--darija-reviews-34030453886e5230/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

851

Index(['review', 'label', 'topic', 'writing_style'], dtype='object')

,review,label,topic,writing_style
0,داسيا فقدت أهم ميزة كانت تميزها وهي السعر للأسف !,negative,automotive,Arabic
1,لاأظنها ستنجح كالنسخ الأولى,negative,automotive,Arabic
2,Khas ykon tghyir mbanch li lfar9 rir fa dizayn lkrasa mikanik tablo fih dak lblstik lmryat 3lach mdyrinch lhm daw volm mzl kif mahwa jwant kifma howa fin kyn lfr9 fdizyn ama ljiti tchof mkynch fra9 fra9 howa y9riban 40% mli kt9ol l3mrya hdi bnsba li ana nas 3morya ama l3morya kolchi daw mzl khsha whd 10 ans wnchofo ana knfdl dacia l9dima,negative,automotive,Arabizi
3,هناك اختيارات أحسن وماركات عالميه أجود من داصيا.,negative,automotive,Arabic


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at otmangi/MorrBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
10,0.857800,0.737201,0.672515,0.548150
20,0.521200,0.620659,0.748538,0.682428
30,0.280600,0.628046,0.766082,0.703291
40,0.147400,0.656563,0.766082,0.706579
50,0.063700,0.790032,0.766082,0.696630
60,0.012600,0.940160,0.795322,0.736419
70,0.022900,0.979153,0.795322,0.733234
80,0.015100,1.119048,0.771930,0.696211
90,0.012400,1.297755,0.742690,0.663988
100,0.002500,1.201116,0.777778,0.717730


,Model,Accuracy,F1
0,SI2M-Lab/DarijaBERT,0.812865,0.755959
3,alger-ia/dziribert,0.812865,0.755253
6,faisalq/EgyBERT,0.766082,0.693996
9,faisalq/SaudiBERT,0.824561,0.752299
12,otmangi/MorRoBERTa,0.771930,0.702497
15,otmangi/MorrBERT,0.812865,0.762196
18,tunis-ai/TunBERT,0.654971,0.568556
